# **Thư viện**

In [20]:
import pandas as pd
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# **Dữ liệu**

In [21]:
data = pd.read_csv("../data/processed/EUR_VND_Exchange_Rate.csv")
data.head()

,Date,Transfer
0,2020-04-01,0.569770
1,2020-04-02,0.569770
2,2020-04-03,0.469007
3,2020-04-04,0.469007
4,2020-04-05,0.469007


In [22]:
timestamps = data["Date"].values  
values = data["Transfer"].fillna(0).astype(np.float32).values

train_size = int(len(values) * 0.8)
train_values, test_values = values[:train_size], values[train_size:]
train_timestamps, test_timestamps = timestamps[:train_size], timestamps[train_size:]


In [23]:
seq_length = 30
def create_sequences(values, timestamps):
        """Tạo sequences với timestamps tương ứng."""
        if len(values) <= seq_length:
            print("Dữ liệu quá ngắn! Không thể tạo sequences.")
            return np.array([]), np.array([]), []

        X, y, y_timestamps = [], [], []
        for i in range(len(values) - seq_length):
            X.append(values[i : i + seq_length])
            y.append(values[i + seq_length])
            y_timestamps.append(pd.to_datetime(timestamps[i + seq_length]).date())

        return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32), np.array(y_timestamps)


X_train, y_train, time_train = create_sequences(train_values, train_timestamps)
X_test, y_test, time_test = create_sequences(test_values, test_timestamps)

In [24]:
X_train.shape

(1431, 30)

In [25]:
X_test.shape

(336, 30)

# **Mô hình LSTM**

In [26]:
def build_lstm_model(hp, seq_length):
    """Xây dựng mô hình LSTM với hyperparameter tuning."""
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))

    model.add(LSTM(
        units=hp.Int("lstm_units_1", 32, 128, 16),
        return_sequences=True,
        activation="tanh",
        kernel_regularizer=l2(hp.Choice("l2_reg", [0.001, 0.01, 0.1]))
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float("dropout_1", 0.1, 0.5, 0.1)))

    model.add(LSTM(
        units=hp.Int("lstm_units_2", 32, 128, 16),
        return_sequences=False,
        activation="tanh",
        kernel_regularizer=l2(hp.Choice("l2_reg", [0.001, 0.01, 0.1]))
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float("dropout_2", 0.1, 0.5, 0.1)))

    model.add(Dense(hp.Int("dense_units", 16, 64, 16), activation="relu"))
    model.add(Dense(1))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])),
        loss="mse",
        metrics=["mae"]
    )

    return model

def train_lstm(X_train, y_train, X_test, y_test, seq_length, model_path, project_name):
    """Huấn luyện và lưu mô hình LSTM sử dụng hyperparameter tuning."""
    tuner = kt.RandomSearch(
        lambda hp: build_lstm_model(hp, seq_length),
        objective="val_loss",
        max_trials=10,
        executions_per_trial=2,
        directory=r"C:\Users\DELL\Downloads\eur-vnd-exchange-rate\models",
        project_name=project_name
    )

    early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True)

    tuner.search(X_train, y_train,
                 epochs=50,
                 batch_size=32,
                 validation_data=(X_test, y_test),
                 callbacks=[early_stopping, model_checkpoint])

    return tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
best_hp = train_lstm(
    X_train, y_train, 
    X_test, y_test, 
    seq_length=30, 
    model_path=r"C:\Users\DELL\Downloads\eur-vnd-exchange-rate\models\best_lstm.h5", 
    project_name="LSTM"
)

Trial 1 Complete [00h 02m 53s]
val_loss: 0.37295906245708466

Best val_loss So Far: 0.37295906245708466
Total elapsed time: 00h 02m 53s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
64                |32                |lstm_units_1
0.001             |0.1               |l2_reg
0.2               |0.4               |dropout_1
48                |80                |lstm_units_2
0.4               |0.4               |dropout_2
32                |64                |dense_units
0.001             |0.0001            |learning_rate

Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.0104 - mae: 0.7276

45/45 ━━━━━━━━━━━━━━━━━━━━ 19s 117ms/step - loss: 1.0036 - mae: 0.7242 - val_loss: 0.4127 - val_mae: 0.5559
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.4112 - mae: 0.4242

45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - loss: 0.4106 - mae: 0.4237 - val_loss: 0.3795 - val_mae: 0.5257
Epoch 3/50
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.2966 - mae: 0.3362

45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 0.2956 - mae: 0.3357 - val_loss: 0.3305 - val_mae: 0.4816
Epoch 4/50
43/45 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.2463 - mae: 0.3025

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.2455 - mae: 0.3016 - val_loss: 0.2480 - val_mae: 0.3924
Epoch 5/50
43/45 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1976 - mae: 0.2523

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1969 - mae: 0.2515 - val_loss: 0.2387 - val_mae: 0.3849
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.1857 - mae: 0.2428

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.1854 - mae: 0.2426 - val_loss: 0.1852 - val_mae: 0.3131
Epoch 7/50
43/45 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1607 - mae: 0.2117

In [ ]:
best_hp.values

NameError: name 'best_hp' is not defined